## Beregningene til inlegg: Korreksjon i aksjemarkedene, dette gjør jeg! 23.02.2022
##### https://www.trifektum.no/finans/korreksjon-i-aksjemarkedene/

In [170]:
import numpy as np
import chart_studio
import chart_studio.plotly as py
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf



In [171]:
df = yf.download("^GSPC", group_by="ticker", period="20y")["Adj Close"].reset_index().rename(columns={'Date': 'dato', "Adj Close": "SP500"}).fillna(method="ffill")
df.tail()

[*********************100%***********************]  1 of 1 completed


,dato,SP500
5031,2022-02-16,4475.009766
5032,2022-02-17,4380.259766
5033,2022-02-18,4348.870117
5034,2022-02-22,4304.759766
5035,2022-02-23,4225.500000


In [172]:
# fra topp
window = 100
latest = df["dato"].max() # timestamp for siste data

df['rollmax'] = df['SP500'].rolling(window, min_periods=1).max() #finn rullerende max
df['Daily_drawdown'] = df['SP500']/df['rollmax'] - 1 #finn drawdown fra nylig topp
df['trend'] = np.where(df['Daily_drawdown'] < 0, 'korrigering', np.nan) # merk nedgangsperioder
df['count'] = df.groupby((df['trend'] != df['trend'].shift(1)).cumsum()).cumcount() #start telling (dager) hver gang ny nedgangsperiode starter
df.loc[df['trend'] != df['trend'].shift(1), 'start'] = df['dato'] #merk hver periode
df['start'].ffill(inplace=True)

current = df["start"].max().strftime("%B %Y") # lagre merkelapp på nåværend korreksjon
df["start"] = df["start"].dt.strftime("%B %Y") # endre datofortmat så det blir lettere å lese


df = df[df['trend']=='korrigering'].groupby(['count', 'start'])['Daily_drawdown'].min().reset_index() 
df = pd.pivot_table(df, values='Daily_drawdown', index=['count'], columns=['start'])
df

start,April 2006,April 2007,April 2010,April 2012,April 2013,April 2014,April 2015,April 2019,April 2021,August 2005,...,September 2003,September 2006,September 2009,September 2012,September 2013,September 2014,September 2017,September 2018,September 2020,September 2021
count,,,,,,,,,,,,,,,,,,,,,
0,-0.001921,-0.002304,-0.016127,-0.003989,-0.010546,-0.001126,-0.004141,-0.006067,-0.007195,-0.007373,...,-0.008210,-0.002263,-0.010069,-0.006148,-0.001843,-0.003073,-0.003046,-0.000369,-0.035126,-0.000335
1,-0.012245,-0.002654,-0.027578,-0.014150,-0.025738,-0.013650,-0.001384,-0.002611,-0.012073,-0.014955,...,-0.020085,-0.005647,-0.019484,-0.004400,-0.009047,-0.009598,-0.002400,-0.003883,-0.042973,-0.003729
2,-0.011391,-0.008727,-0.021293,-0.014771,-0.011799,-0.024253,-0.005119,-0.002573,-0.011115,-0.017598,...,-0.014753,-0.003563,-0.025456,-0.003220,-0.013723,-0.014214,-0.004617,-0.005183,-0.069537,-0.005043
3,-0.019054,-0.006099,-0.008626,-0.025961,-0.025958,-0.020593,-0.015196,-0.000819,-0.012063,-0.010972,...,-0.029050,NaN,-0.008100,-0.003759,-0.016285,-0.006493,-0.004545,-0.008455,-0.050793,-0.009625
4,-0.017872,NaN,-0.025130,-0.042599,-0.032485,-0.009900,-0.004439,NaN,-0.002339,-0.012779,...,-0.034928,NaN,-0.010311,-0.003834,-0.018980,-0.022557,-0.000478,-0.005715,-0.067484,-0.017274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
df = df.loc[:, (df.min()<-.05)] # filtrere korreksjoner <5% for å spare lagringsplass
df.min()

start
April 2010       -0.159947
April 2012       -0.099363
August 2005      -0.054777
February 2004    -0.075910
February 2007    -0.058616
February 2011    -0.064132
February 2020    -0.339250
January 2010     -0.081279
January 2014     -0.057613
January 2018     -0.101595
January 2022     -0.119056
July 2007        -0.094251
July 2019        -0.061226
June 2009        -0.070893
June 2016        -0.055957
March 2002       -0.318374
March 2005       -0.071664
May 2006         -0.076990
May 2011         -0.187693
May 2013         -0.057556
May 2015         -0.133051
May 2019         -0.068361
October 2007     -0.423559
October 2009     -0.056216
September 2012   -0.076711
September 2014   -0.074015
September 2018   -0.197782
September 2020   -0.096045
September 2021   -0.052125
dtype: float64

In [175]:
for col in df.loc[:, df.columns != current].columns: # stop etter bunn
    df[col] = df[col].mask(lambda x: x.shift(1).eq(df[col].min()).cummax())
    
df.head(20)

/Users/familien_nerva-koch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/familien_nerva-koch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/familien_nerva-koch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

start,April 2010,April 2012,August 2005,February 2004,February 2007,February 2011,February 2020,January 2010,January 2014,January 2018,...,May 2013,May 2015,May 2019,October 2007,October 2009,September 2012,September 2014,September 2018,September 2020,September 2021
count,,,,,,,,,,,,,,,,,,,,,
0,-0.016127,-0.003989,-0.007373,-0.004880,-0.001404,-0.020529,-0.005401,-0.010823,-0.008862,-0.006732,...,-0.009308,-0.002234,-0.007502,-0.003217,-0.008098,-0.006148,-0.003073,-0.000369,-0.035126,-0.000335
1,-0.027578,-0.014150,-0.014955,-0.010322,-0.008248,-0.026515,-0.014293,-0.029342,-0.009192,-0.017557,...,-0.011173,-0.012493,-0.009610,-0.006862,-0.015038,-0.004400,-0.009598,-0.003883,-0.042973,-0.003729
2,-0.021293,-0.014771,-0.017598,-0.000665,-0.011483,-0.027483,-0.047328,-0.050833,-0.011681,-0.017077,...,-0.011719,-0.003445,-0.000065,-0.002140,-0.004554,-0.003220,-0.014214,-0.005183,-0.069537,-0.005043
3,-0.008626,-0.025961,-0.010972,-0.005131,-0.007063,-0.017223,-0.076175,-0.046469,-0.005670,-0.017714,...,-0.005452,-0.004707,-0.004535,-0.010504,-0.016677,-0.003759,-0.006493,-0.008455,-0.050793,-0.009625
4,-0.025130,-0.042599,-0.012779,-0.009242,-0.041543,-0.011757,-0.079666,-0.050477,-0.010777,-0.038547,...,-0.012461,-0.010996,-0.020972,-0.017008,-0.028199,-0.003834,-0.022557,-0.005715,-0.067484,-0.017274
5,-0.012339,-0.035468,-0.005807,-0.011790,-0.036213,-0.027312,-0.120311,-0.045843,-0.031428,-0.077947,...,-0.008837,-0.008959,-0.022544,-0.015277,-0.031423,-0.006058,-0.014174,-0.005722,-0.066987,-0.015037
6,-0.035883,-0.022177,-0.011767,-0.014485,-0.038714,-0.025741,-0.127558,-0.057119,-0.036151,-0.061865,...,-0.023018,-0.009959,-0.025497,-0.016018,-0.050350,-0.016496,-0.016685,-0.002102,-0.055099,-0.020697
7,-0.042234,-0.034375,-0.008972,-0.016126,-0.049675,-0.008965,-0.087391,-0.066387,-0.030232,-0.066557,...,-0.017218,-0.007861,-0.021872,-0.041223,-0.028964,-0.022139,-0.019425,-0.002498,-0.050167,-0.012398
8,-0.073221,-0.034862,-0.020642,-0.012170,-0.058616,-0.016277,-0.113043,-0.053068,-0.040132,-0.101595,...,-0.022634,-0.016416,-0.045474,-0.037581,-0.056216,-0.012703,-0.032416,-0.001788,-0.054554,-0.013930


In [6]:
graphdata = df.reset_index().melt(id_vars=["count"])
graphdata["color"] = graphdata["start"] == current

In [104]:
fig = go.Figure()

d = graphdata[graphdata["color"]==False]

for peak in d.start.unique():
    temp = d[d['start']==peak].dropna()
    fig.add_trace(go.Scatter(x=temp['count'], 
                             y=temp['value'], 
                             mode='lines',
                            name=peak,
                            hovertemplate = '%{y} etter %{x} dager',
                            line=dict(color='rgba(60, 0, 255, .3)', width=1),
                            connectgaps=True,
                            ))

fig.add_trace(go.Scatter(x=graphdata[graphdata["color"]==True].dropna()['count'], 
                             y=graphdata[graphdata["color"]==True].dropna()['value'], 
                             mode='lines',
                            name=current,
                             hovertemplate = '%{y} etter %{x} dager',
                            line=dict(color='rgb(60, 0, 255)', width=3),
                            connectgaps=True,
                            ))

latest_x = graphdata[graphdata["color"]==True].dropna().iloc[-1]['count']
latest_y = graphdata[graphdata["color"]==True].dropna().iloc[-1]['value']

fig.add_trace(go.Scatter(
        x=[latest_x],
        y=[latest_y],
        mode='markers',
        hovertemplate='%{y} etter %{x} dager',
        name=latest.strftime('%d.%m.%Y'),
        marker=dict(
            color='rgba(60, 0, 255, 1)',
            size=9,
            line=dict(
                color='rgba(60, 0, 255, .3)',
                width=6
            )
    )))

    
fig.update_layout(
    xaxis=dict(
        fixedrange=True,
        range=[0,300],
        #title="antall handledager etter topp",
        showline=False,
        showgrid=False,
        showticklabels=True,
        linecolor='rgba(204, 204, 204, 0)',
        linewidth=None,
        ticks=None,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(112, 133, 153)',
        ),
    ),
    yaxis=dict(
        fixedrange=True,
        showgrid=True,
        gridcolor='rgba(112, 133, 153, .2)',
        gridwidth=.5,
        zeroline=False,
        showline=True,
        showticklabels=True,
        side="right",
        tickformat='.0%',
        tickvals=[0, -.1, -.15, -.2,-.3,-.4],
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(112, 133, 153)'),
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=0, r=0, t=0, b=0

    ),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
)

#py.plot(fig, filename='SPX_korreksjoner.html', auto_open=False, link=True)    
fig.show()

## Lag drawdown graf

In [105]:
duration = pd.DataFrame()

for c in df.columns:
    d = {'index': c, 'varighet': df[c].dropna().reset_index()["count"].max()}
    duration = duration.append(d, ignore_index=True)

duration

,index,varighet
0,April 2010,48.0
1,April 2012,41.0
2,August 2005,49.0
3,February 2004,125.0
4,February 2007,8.0
5,February 2011,16.0
6,February 2020,22.0
7,January 2010,13.0
8,January 2014,11.0
9,January 2018,8.0


In [106]:
df2 = graphdata.groupby(["start"])["value"].min().reset_index()
df2["value"] = df2["value"]
df2["color"] = df2["start"] == current
df2 = df2.merge(duration, how="left", left_on="start", right_on="index").drop("index", axis=1)
df2

,start,value,color,varighet
0,April 2010,-0.159947,False,48.0
1,April 2012,-0.099363,False,41.0
2,August 2005,-0.054777,False,49.0
3,February 2004,-0.075910,False,125.0
4,February 2007,-0.058616,False,8.0
5,February 2011,-0.064132,False,16.0
6,February 2020,-0.339250,False,22.0
7,January 2010,-0.081279,False,13.0
8,January 2014,-0.057613,False,11.0
9,January 2018,-0.101595,False,8.0


In [169]:
fig = go.Figure()

d2 = df2[df2["color"]==False]

fig = go.Figure(data=[go.Bar(
            x=d2.start, 
            y=d2.value,
            text=d2.value,
            textposition='inside',
            hovertemplate = 'Etter toppen %{x} var <br>S&P på det meste %{y}',
            texttemplate='%{text:.1%}',
            marker_color='#F1F5FE',
            marker_line_color = '#6e7a91',
            marker_line_width=1,
        )])

fig.add_trace(go.Bar(
    x=df2[df2["color"]==True]['start'],
    y=df2[df2["color"]==True]['value'],
    text=df2[df2["color"]==True]['value'],
    textposition='inside',
    texttemplate='%{text:.1%}',
    hovertemplate = 'Foreløpig bunn er <br>%{y} fra toppen i januar',
    #name=df2[df2["color"]==True]['start'],
    marker_color='rgba(60, 0, 255, 1)',
    marker_line_color = 'rgba(60, 0, 255, .3)',
    marker_line_width=8,
))

fig.update_layout(
    xaxis=dict(
        fixedrange=True,
        #title="antall handledager etter topp",
        showline=False,
        showgrid=False,
        showticklabels=True,
        tickangle=-90,
        linecolor='rgba(204, 204, 204, 0)',
        linewidth=None,
        ticks=None,
        categoryorder='total descending',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(112, 133, 153)',
        ),
    ),
    yaxis=dict(
        fixedrange=True,
        showgrid=True,
        gridcolor='rgba(112, 133, 153, .2)',
        gridwidth=.5,
        zeroline=False,
        showline=True,
        showticklabels=True,
        side="right",
        tickformat='.0%',
        tickvals=[0, -.1, -.15, -.2,-.3,-.4],
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(112, 133, 153)'),
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=10, r=0, t=0, b=0

    ),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    bargap=0,
)

fig.show()

In [107]:
df2

,start,value,color,varighet
0,April 2010,-0.159947,False,48.0
1,April 2012,-0.099363,False,41.0
2,August 2005,-0.054777,False,49.0
3,February 2004,-0.075910,False,125.0
4,February 2007,-0.058616,False,8.0
5,February 2011,-0.064132,False,16.0
6,February 2020,-0.339250,False,22.0
7,January 2010,-0.081279,False,13.0
8,January 2014,-0.057613,False,11.0
9,January 2018,-0.101595,False,8.0


In [35]:
hover_data={'color':False, 
            'value':':.0f'}


fig = px.bar(df2.sort_values("varighet", ascending=True), color="color", x='start', y='varighet', hover_name="start", text="varighet", hover_data=hover_data,
            color_discrete_sequence=["#343a40", "#dc3545"],
             #range_x=[0,500]
             )
fig.update_traces(texttemplate='%{text:.0} dager', textposition='auto', hovertemplate='%{text:.0} dager')


fig.add_shape(type='rect', layer="below",
                x0=-0.5, y0=df2["varighet"].mean()-df2["varighet"].std()*0.5,
                x1=51.5, y1=df2["varighet"].mean()+df2["varighet"].std()*0.5,
                line=dict(color='rgba(220,53,69,0)',),
                xref='x', yref='y', fillcolor= "rgba(253,126,20,0.2)")

fig.add_shape(type='line', layer="below",
                x0=-0.5, y0=df2["varighet"].mean(),
                x1=51.5, y1=df2["varighet"].mean(),
                line=dict(color= "#fd7e14",),
                xref='x', yref='y')




fig.update_layout(showlegend=False,
                 #hoverlabel = dict(bgcolor = "#aab2bd"),
                  xaxis=dict(title="",
                            showline=True,
                            showgrid=False,
                            showticklabels=True,
                            zeroline=False,
                            tickangle=-90,
                            linecolor="#6c757d",
                            linewidth=1,
                            ticks="outside",
                            tickfont=dict(family="Arial", size=12, color="#6c757d"),
                            titlefont=dict(family="Arial", size=12, color="rgb(150,150,150)"),),
                  yaxis=dict(title="",
                            side="right",
                            showgrid=False,
                            showticklabels=False,
                            tickfont=dict(family="Arial", size=12, color="rgba(150,150,150,0.6)")),
                  plot_bgcolor="rgba(0,0,0,0)", paper_bgcolor='rgba(0, 0, 0, 0)',
                  margin=dict(l=0, r=0, t=0, b=0),
                 )


fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total ascending'})

annotations = []

annotations.append(dict(xref="paper", yref="paper", x=0, y=1,
                       xanchor="left", yanchor="top", align="left",
                       text="Oppdatert: "+latest.strftime('%d.%m.%Y'),
                       font=dict(family="Arial",
                                size=12,
                                color="#6c757d"),
                       showarrow=False))
annotations.append(dict(x=2, y=df2["varighet"].mean(),
                       xanchor="left", yanchor="bottom", align="left",
                       text="I gjennomsnittlig tar det "+str(round(df2["varighet"].mean()))+" handledager til S&P 500 topper (+/- 0,5 standardavvik)",
                       font=dict(family="Arial",
                                size=12,
                                color="#fd7e14"),
                       showarrow=False))

fig.update_layout(annotations=annotations)

fig.show()

In [87]:
pd.DataFrame(tab.iloc[:, 1:].count(1).replace(0, np.nan).dropna()).reset_index()

,count,0
0,0,31.0
1,1,31.0
2,2,31.0
3,3,24.0
4,4,20.0
...,...,...
172,172,1.0
173,173,1.0
174,174,1.0
175,175,1.0


In [93]:
probability = pd.DataFrame(tab.iloc[:, 1:].count(1).replace(0, np.nan).dropna()).reset_index()
probability[0] = (31-probability[0])/30

fig = px.line(probability, x="count", y=0, color_discrete_sequence=["rgb(37,37,37)"],
             labels = {"count": "Antall handledager ", "0": "Sannsynlighet for at bunnen er tilbakelagt* "})

fig.update_layout(showlegend=False,
                 #hoverlabel = dict(bgcolor = "#aab2bd"),
                  xaxis=dict(title="# handledager etter brudd",
                            showline=True,
                            showgrid=False,
                            showticklabels=True,
                            linecolor="rgba(204,204,204, 0.7)",
                            linewidth=2,
                            ticks="outside",
                            tickfont=dict(family="Arial", size=12, color="rgb(150,150,150)"),
                            titlefont=dict(family="Arial", size=12, color="rgb(150,150,150)"),),
                  yaxis=dict(title="Sannsynlighet for at bunnen tilbakelagt*",
                            side="right",
                            showticklabels=True,
                             tickformat = ",.0%", hoverformat = ",.0%",
                            linecolor="rgba(204,204,204, 0.7)",
                            linewidth=2,
                            tickfont=dict(family="Arial", size=12, color="rgba(150,150,150,1)"),
                            titlefont=dict(family="Arial", size=12, color="rgb(150,150,150)"),),
                  plot_bgcolor="rgba(0,0,0,0)", paper_bgcolor='rgba(0, 0, 0, 0)',
                  margin=dict(l=50, r=50, t=80),
                 )

annotations = []
annotations.append(dict(xref="paper", yref="paper", x=0.0, y=1.09,
                       xanchor="left", yanchor="bottom",
                       text="S&P 500 bunner innen 17 handledager 83% av gangene",
                       font=dict(family="Arial",
                                size=22,
                                color="rgb(37,37,37)"),
                       showarrow=False))

annotations.append(dict(xref="paper", yref="paper", x=0, y=1.09,
                       xanchor="left", yanchor="top", align="left",
                       text="Linjen illustrerer fordeling av laveste punkt atter antall dager siden brudd gjennom 50 dagers glidende gjennomsnitt.<br>"+"Basert på 10 års historikk for S&P 500 i USD "+
                        "* Fortusetter historisk fordeling",
                       font=dict(family="Arial",
                                size=12,
                                color="rgba(82,82,82,0.9)"),
                       showarrow=False))

fig.update_layout(annotations=annotations)

#py.plot(fig, filename='SPX_SMA50_probability.html', auto_open=False, link=True)
fig.show()

In [41]:
low = df.loc[(df["start"]==fokus)]
low = low[low["value"]== low["value"].min()]["count"]

df.loc[(df["start"]==fokus)]["count"] 

7812      0
7813      1
7814      2
7815      3
7816      4
       ... 
8059    247
8060    248
8061    249
8062    250
8063    251
Name: count, Length: 252, dtype: int64

In [42]:
df.groupby(["start", "value"])["count"].min()

start           value      
April 2012      4136.616862    38
                4192.222941    29
                4223.424660    28
                4241.129249    37
                4250.806915    36
                               ..
September 2021  4432.990234     1
                4443.109863     7
                4448.979980     5
                4455.479980     6
                4473.750000     0
Name: count, Length: 513, dtype: int64

In [43]:
import numpy as np
import chart_studio
import pandas as pd
import plotly.express as px
from fredapi import Fred

chart_studio.tools.set_credentials_file(username='oystein.nerva', api_key='0ODcZ6UisJ8n0AjEQt9N')
chart_studio.tools.set_config_file(world_readable=True, sharing='public')
fred = Fred(api_key='005732749dbeb573e7e685e466a2695f')
fokus = "September 2020"

df = fred.get_series('SP500').reset_index().rename(columns={'index': 'dato', 0: "SP500"}).fillna(method="ffill")

df["sma(50)"] = df["SP500"].rolling(50).mean()
df["trend"] = np.where(df['SP500'].shift(-1) < df["sma(50)"].shift(-1), "bear", (df["SP500"]>=df["SP500"].rolling(100).max()).replace([True, False], ["bull", np.nan]))
df["trend"].ffill(inplace=True)
df['count'] = df.groupby((df['trend'] != df['trend'].shift(1)).cumsum()).cumcount()
df.loc[df['trend'] != df["trend"].shift(1), 'start'] = df["dato"]
df["start"].ffill(inplace=True)
df["start"] = df["start"].dt.strftime("%B %Y")
latest = df["dato"].max()
df = df[df["trend"]=="bear"].groupby(["count", "start"])["SP500"].min().reset_index()
df = pd.pivot_table(df, values='SP500', index=["count"], columns=['start'])

for col in df.copy().loc[:, df.columns != fokus].columns: # stop etter bunn
    df[col] = df[col].mask(lambda x: x.shift(1).eq(df[col].min()).cummax())

startverdi = df.iloc[0][fokus]

df = df.pct_change(periods=1, limit=1)
tab = df.copy()
df = np.exp(np.log1p(df).cumsum())*startverdi
df.iloc[:1].replace(np.nan, startverdi, inplace=True)
df = df.reset_index().melt(id_vars=["count"])
df["color"] = df["start"] == fokus

hover_data={'color':False,
            'value':':.0f'}

fig = px.line(df, x="count", y="value", color="color", hover_name="start", hover_data=hover_data,
              labels = {"count": "Antall handledager", "value": "S&P i dagens verdi"}, color_discrete_sequence=["rgba(82,82,82,0.3)", "#fc5661"])

fig.update_layout(showlegend=False,
                 #hoverlabel = dict(bgcolor = "#aab2bd"),
                  xaxis=dict(title="# handledager etter brudd",
                            showline=True,
                            showgrid=False,
                            showticklabels=True,
                            linecolor="rgba(204,204,204, 0.7)",
                            linewidth=2,
                            ticks="outside",
                            tickfont=dict(family="Arial", size=12, color="rgb(150,150,150)"),
                            titlefont=dict(family="Arial", size=12, color="rgb(150,150,150)"),),
                  yaxis=dict(title="",
                            side="right",
                            showticklabels=True,
                            tickfont=dict(family="Arial", size=12, color="rgba(150,150,150,0.6)")),
                  plot_bgcolor="rgba(0,0,0,0)", paper_bgcolor='rgba(0, 0, 0, 0)',
                  margin=dict(l=50, r=50, t=80),
                 )

annotations = []
annotations.append(dict(xref="paper", yref="paper", x=0.0, y=1.09,
                       xanchor="left", yanchor="bottom",
                       text="Hvor langt pleier S&P 500 falle?",
                       font=dict(family="Arial",
                                size=22,
                                color="rgb(37,37,37)"),
                       showarrow=False))

annotations.append(dict(xref="paper", yref="paper", x=0, y=1.09,
                       xanchor="left", yanchor="top", align="left",
                       text="Rød linje viser utvikling i S&P 500 etter indeksen brøt ned gjennom 50 dagers glidende gjennomsnitt "+fokus+".<br>Grå linjer viser utvikling i S&P 500 ved tidligere brudd siste 10 år (rebalansert til dagens utgangspunkt). Hensikten er å illustrere hvor langt ned <br>"+
                        "indeksen kan falle om historien gjentar seg. Oppdatert tom.: "+latest.strftime('%d.%m.%Y'),
                       font=dict(family="Arial",
                                size=12,
                                color="rgba(82,82,82,0.9)"),
                       showarrow=False))

fig.update_layout(annotations=annotations)

fig.show()

/Users/familien_nerva-koch/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4379: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

